In [30]:
import geopandas as gpd
import momepy as mm
import os
import pickle
import libpysal

In [31]:
local_crs = 27700
place = "test_OS"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=1

In [33]:
buildings = gpd.read_parquet(f"../output/{place}/buildings_morphometric_p3.pq")
tess = gpd.read_parquet(f"../output/{place}/tessellation_morphometric_p3.pq")

In [34]:
buildings["building_corners"] = mm.Corners(buildings).series
buildings["buildings_shared_walls_ratio"] = mm.SharedWallsRatio(buildings).series

  0%|          | 912/828537 [00:00<02:59, 4598.87it/s]c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\momepy\shape.py:826: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
100%|██████████| 828537/828537 [02:48<00:00, 4926.86it/s]
c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\momepy\distribution.py:137: FutureWarning: The `query_bulk()` method is deprecated and will be removed in GeoPandas 1.0. You can use the `query()` method instead.
  inp, res = gdf.sindex.query_bulk(gdf.geometry, predicate="intersects")


In [35]:
cencon = mm.CentroidCorners(buildings)
buildings['buildings_centroidcornermean'] = cencon.mean
buildings['buildings_centroidcornerstd'] = cencon.std

  0%|          | 1098/828537 [00:00<06:34, 2099.81it/s]c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\momepy\shape.py:1166: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
100%|██████████| 828537/828537 [06:42<00:00, 2060.37it/s]


In [36]:
tess["tess_orientation"]

0         42.157345
1         40.827094
2         33.073745
3         38.347293
4         27.527727
            ...    
828532    30.393956
828533     5.330012
828534    19.326235
828535    39.036477
828536    39.451461
Name: tess_orientation, Length: 828537, dtype: float64

In [37]:
buildings['building_cell_alignment'] = mm.CellAlignment(buildings, tess, "building_orientation", "tess_orientation", 'uID', 'uID').series

In [38]:
tess["tess_area_ratio"] = mm.AreaRatio(tess, buildings, "tess_area", "building_area", 'uID').series
tess["tess_circularCompactness"] = mm.CircularCompactness(tess, "tess_area").series

In [39]:
file_path = f"../output/{place}/queen_1.pkl"

if not os.path.exists(file_path):
    result = "File not found."
else:
    # Load the pickle file
    with open(file_path, 'rb') as file:
        queen_1 = pickle.load(file)
    result = "File loaded successfully."


In [40]:
file_path = f"../output/{place}/queen_3.pkl"

if not os.path.exists(file_path):
    result = "File not found."
else:
    # Load the pickle file
    with open(file_path, 'rb') as file:
        queen_3 = pickle.load(file)
    result = "File loaded successfully."


In [41]:
buildings["building_alignment"] = mm.Alignment(buildings, queen_1, "uID", "building_orientation").series

100%|██████████| 828537/828537 [00:21<00:00, 39253.02it/s]  


In [42]:
blg_q1 = libpysal.weights.contiguity.Queen.from_dataframe(buildings)
buildings["building_perimeter_wall"] = mm.PerimeterWall(buildings, blg_q1).series

C:\Users\reube\AppData\Local\Temp\ipykernel_6304\3326318115.py:1: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  blg_q1 = libpysal.weights.contiguity.Queen.from_dataframe(buildings)
c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\libpysal\weights\weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 424366 disconnected components.
 There are 243650 islands with ids: 0, 4, 6, 7, 8, 10, 11, 12, 14, 17, 18, 19, 21, 22, 24, 26, 28, 31, 32, 39, 41, 42, 47, 48, 49, 51, 52, 54, 59, 60, 62, 63, 69, 73, 76, 77, 81, 82, 83, 84, 90, 93, 95, 96, 99, 100, 101, 103, 105, 107, 108, 109, 110, 113, 115, 116, 118, 119, 120, 121, 122, 123, 125, 127, 131, 135, 136, 137, 140, 141, 142, 147, 149, 150, 158, 160, 163, 164, 168, 169, 170, 174, 178, 181, 182, 187, 190, 191, 197, 198, 200, 202, 203, 205, 207, 210, 223, 225, 227, 228, 232, 234, 240, 242, 245, 246,

In [43]:
streets = gpd.read_parquet(f"../output/{place}/streets_p2.pq").to_crs(local_crs).explode()

C:\Users\reube\AppData\Local\Temp\ipykernel_6304\3721643334.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  streets = gpd.read_parquet(f"../output/{place}/streets_p2.pq").to_crs(local_crs).explode()


In [44]:
blocks0 = mm.Blocks(tess, streets, buildings, id_name='bID', unique_id='uID')

c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\momepy\elements.py:619: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  weights = libpysal.weights.Queen.from_dataframe(cut, silence_warnings=True)


In [45]:
buildings['bID'] = blocks0.buildings_id
tess['bID'] = blocks0.tessellation_id

In [46]:
blocks = blocks0.blocks
blocks["block_area"] = mm.Area(blocks).series
blocks["block_circularcompactness"] = mm.CircularCompactness(blocks, "block_area").series
blocks["block_orientation"] = mm.Orientation(blocks).series
 
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID")
 
blocks["block_Neighbors"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True).series
blocks["block_count"] = mm.Count(blocks, buildings, "bID", "bID", weighted=True).series

100%|██████████| 10813/10813 [00:00<00:00, 21682.20it/s]
c:\Users\reube\anaconda3\envs\nbprocessor\Lib\site-packages\libpysal\weights\weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 14 disconnected components.
 There are 8 islands with ids: 1257, 1518, 2970, 8273, 9245, 10112, 10113, 10426.
  warnings.warn(message)
100%|██████████| 10813/10813 [00:00<00:00, 86108.35it/s]


In [47]:
buildings["building_adjacency"] = mm.BuildingAdjacency(buildings, queen_3, 'uID', blg_q1).series

Calculating adjacency: 100%|██████████| 828537/828537 [00:08<00:00, 93343.16it/s] 


In [48]:
tess["tess_block_count"] = mm.BlocksCount(tess, 'bID', queen_3, 'uID').series

100%|██████████| 828537/828537 [20:39<00:00, 668.27it/s]


In [49]:
buildings.to_parquet(f"../output/{place}/buildings_morphometric_p3.pq")
tess.to_parquet(f"../output/{place}/tessellation_morphometric_p3.pq")
blocks.to_parquet(f"../output/{place}/blocks_morphometric_p4.5.pq")